In [1]:
#libraries
import os
#The OS: functions for interacting with the operating system
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
import numpy as np
import datetime
from IPython.display import display

In [2]:
# source
user_credential = r"/Users\dinhhoang.nguyen.CONCENTRIX\OneDrive - Concentrix Corporation\WFM-internal"

In [3]:
time_reset = dt(year=2023, month=8, day=1)
update_less = True

In [4]:
# IMPORT EPS
eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps_full.json'))
if update_less is True:
    eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps.json'))
else:
    eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps_full.json'))

In [5]:
# IMPORT SCHEDULE
schedule = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\schedule_full.json'))
schedule['Week'] = schedule['Date'].dt.strftime('%Y%W')
display(schedule)

,Emp ID,Name,Date,Shift,LOB,Shift_type,Week
0,101754262,Nguyen Diem Xuan Nhi,2022-09-19,OFF,EN,OFF,202238
1,101754262,Nguyen Diem Xuan Nhi,2022-09-20,1100-2000,EN,DS,202238
2,101754262,Nguyen Diem Xuan Nhi,2022-09-21,1100-2000,EN,DS,202238
3,101754262,Nguyen Diem Xuan Nhi,2022-09-22,1100-2000,EN,DS,202238
4,101754262,Nguyen Diem Xuan Nhi,2022-09-23,1100-2000,EN,DS,202238
...,...,...,...,...,...,...,...
152909,102003080,Doan Ai Khue,2024-01-03,1300-2200,Senior VICSP,DS,202401
152910,102003080,Doan Ai Khue,2024-01-04,OFF,Senior VICSP,OFF,202401
152911,102003080,Doan Ai Khue,2024-01-05,OFF,Senior VICSP,OFF,202401
152912,102003080,Doan Ai Khue,2024-01-06,1300-2200,Senior VICSP,DS,202401


In [6]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\masterroster.json'))
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

In [7]:
# IMPORT TL
tl = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\tl.json'))

In [8]:
# IMPORT CUIC
cuic = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\cuic.json'))

In [9]:
#IMPORT CPI
cpi = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\cpi.json'))
cpi['Employee_ID'] = cpi['Employee_ID'].astype("Int64")
cpi['Date'] = cpi['Date'].dt.strftime('%Y-%m-%d')
cpi['Date'] = pd.to_datetime(cpi['Date'], format = "mixed")

In [10]:
# cuic
agent_wise_cuic = cuic
agent_wise_cuic = agent_wise_cuic.groupby(['Employee_ID', 'LoginName', 'Session Date'], as_index=False)[['AgentAvailTime']].sum()
agent_wise_cuic['AgentAvailTime']=agent_wise_cuic['AgentAvailTime']*24
agent_wise_cuic['Session Date'] = pd.to_datetime(agent_wise_cuic['Session Date'], format='mixed')
agent_wise_cuic['Employee_ID'] = agent_wise_cuic['Employee_ID'].astype("int64")

In [11]:
# add session date in EPS
aw_eps = eps

aw_eps['Session login date'] = pd.to_datetime(aw_eps['Session login date'], format='mixed').dt.date
aw_eps['Session login date-1'] = aw_eps['Session login date'] - pd.Timedelta(days=1)
aw_eps['Session login date-1'] = pd.to_datetime(aw_eps['Session login date-1'], format = "mixed")


aw_eps = pd.merge(aw_eps, schedule[['Emp ID', 'Date', 'Shift','Shift_type']], left_on=['EID', 'Session login date-1'], right_on=['Emp ID', 'Date'], how='left')

def session_date(x):
    if x['Shift_type'] == 'NS' and x['Session login time'] <= t(12, 0, 0):
        return x['Session login date-1']
    else:
        return x['Session login date']
    
aw_eps['Session login time'] = pd.to_datetime(aw_eps['Session login time'], format='mixed').dt.time
aw_eps['Session date eps'] = aw_eps.apply(session_date, axis=1)
aw_eps['Session date eps'] = pd.to_datetime(aw_eps['Session date eps'], format = 'mixed')

aw_eps.to_excel(os.path.join(r'C:', user_credential, r'DB\filecsv\aw_eps.xlsx'),sheet_name='sheet_1')

display(aw_eps)

,EID,Session Login,Session Logout,BPE Code,Total Time,Session login date,Session login time,Session logout date,Session logout time,Session login date-1,Emp ID,Date,Shift,Shift_type,Session date eps
0,102168291,2023-07-31 07:03:00 PM,2023-07-31 11:59:00 PM,Break,217,2023-08-01,00:03:00,2023-08-01,04:59:00,2023-07-31,102168291.0,2023-07-31,2200-0700,NS,2023-07-31
1,102168291,2023-07-31 07:03:00 PM,2023-07-31 11:59:00 PM,Lunch,3601,2023-08-01,00:03:00,2023-08-01,04:59:00,2023-07-31,102168291.0,2023-07-31,2200-0700,NS,2023-07-31
2,102168291,2023-07-31 07:03:00 PM,2023-07-31 11:59:00 PM,Picklist - off Phone,4920,2023-08-01,00:03:00,2023-08-01,04:59:00,2023-07-31,102168291.0,2023-07-31,2200-0700,NS,2023-07-31
3,102168291,2023-07-31 07:03:00 PM,2023-07-31 11:59:00 PM,Ready or Talking,9049,2023-08-01,00:03:00,2023-08-01,04:59:00,2023-07-31,102168291.0,2023-07-31,2200-0700,NS,2023-07-31
4,102168231,2023-07-31 09:56:00 PM,2023-07-31 11:56:00 PM,Not Working Yet,6,2023-08-01,02:56:00,2023-08-01,04:56:00,2023-07-31,102168231.0,2023-07-31,2200-0700,NS,2023-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356718,101631001,2024-01-01 03:20:00 AM,2024-01-01 10:11:00 AM,Break,2043,2024-01-01,09:20:00,2024-01-01,16:11:00,2023-12-31,101631001.0,2023-12-31,OFF,OFF,2024-01-01
356719,101631001,2024-01-01 03:20:00 AM,2024-01-01 10:11:00 AM,Lunch,3528,2024-01-01,09:20:00,2024-01-01,16:11:00,2023-12-31,101631001.0,2023-12-31,OFF,OFF,2024-01-01
356720,101631001,2024-01-01 03:20:00 AM,2024-01-01 10:11:00 AM,Meeting,1809,2024-01-01,09:20:00,2024-01-01,16:11:00,2023-12-31,101631001.0,2023-12-31,OFF,OFF,2024-01-01
356721,101631001,2024-01-01 03:20:00 AM,2024-01-01 10:11:00 AM,Picklist - off Phone,7200,2024-01-01,09:20:00,2024-01-01,16:11:00,2023-12-31,101631001.0,2023-12-31,OFF,OFF,2024-01-01


In [12]:
# Agent wise frame
agent_wise_frame = aw_eps
agent_wise_frame['Location'] = 'HCM'
agent_wise_frame['Country'] = 'Vietnam'
agent_wise_frame['Account'] = 'Booking.com'
agent_wise_frame['Work Status'] = 'WFO'


agent_wise_frame['Staff hour (In hours)'] = (agent_wise_frame.apply(lambda x: x['Total Time']/3600, axis=1))
agent_wise_frame['Total break (In hours)'] = (agent_wise_frame.apply(lambda x: x['Total Time']/3600 if x['BPE Code'] == 'Break' or x['BPE Code'] == 'Lunch' else 0, axis=1))
agent_wise_frame['THT (In hours)'] = (agent_wise_frame.apply(lambda x: x['Total Time']/3600 if x['BPE Code'] == 'Picklist - off Phone' or x['BPE Code'] == 'Ready or talking' else 0, axis=1))
agent_wise_frame['Other Productive Aux(In Hours)'] = (agent_wise_frame.apply(lambda x: x['Total Time']/3600 if x['BPE Code'] == 'Meeting' or x['BPE Code'] == 'Training' else 0, axis=1))
agent_wise_frame['Non Productive Aux(In Hours)'] = (agent_wise_frame.apply(lambda x: x['Total Time']/3600 if x['BPE Code'] == 'Break' else 0, axis=1))


# add cuic to main frame
agent_wise_frame = pd.merge(agent_wise_frame, agent_wise_cuic, left_on =['Emp ID', 'Session date eps'], right_on=['Employee_ID', 'Session Date'], how='left')
agent_wise_frame['Avail Time(In Hours)'] = (agent_wise_frame['AgentAvailTime'].fillna(0))
# agent_wise_frame = agent_wise_frame.replace('', np.nan).dropna()



agent_wise_frame = agent_wise_frame.groupby(['Session Date','Location','Country','Account','Work Status'],as_index=False)[['Staff hour (In hours)','Total break (In hours)','THT (In hours)','Avail Time(In Hours)','Other Productive Aux(In Hours)','Non Productive Aux(In Hours)']].sum()
agent_wise_frame['Session Date'] = pd.to_datetime(agent_wise_frame['Session Date'], format = 'mixed').dt.strftime('%m/%d/%Y')


agent_wise_frame = agent_wise_frame.sort_values(by=['Session Date'], ascending=False)

agent_wise_frame.to_json(os.path.join(r'C:', user_credential, r'DB\filejson\agentwiserp.json'), orient='records')
agent_wise_frame.to_excel(os.path.join(r'C:', user_credential, r'DB\filecsv\agentwiserp.xlsx'),sheet_name='sheet_1') 

display(agent_wise_frame)

,Session Date,Location,Country,Account,Work Status,Staff hour (In hours),Total break (In hours),THT (In hours),Avail Time(In Hours),Other Productive Aux(In Hours),Non Productive Aux(In Hours)
152,12/31/2023,HCM,Vietnam,Booking.com,WFO,823.150556,130.790833,468.103889,294.823055,47.346944,43.156667
151,12/30/2023,HCM,Vietnam,Booking.com,WFO,1552.125833,248.144722,965.961944,377.668611,83.591111,82.045556
150,12/29/2023,HCM,Vietnam,Booking.com,WFO,2830.811667,455.764722,1606.661667,921.545277,252.215833,149.185833
149,12/28/2023,HCM,Vietnam,Booking.com,WFO,2764.000556,448.060278,1559.339444,916.921388,275.978889,145.033889
148,12/27/2023,HCM,Vietnam,Booking.com,WFO,2873.514722,460.706111,1604.198611,1098.915277,284.865000,148.208611
...,...,...,...,...,...,...,...,...,...,...,...
2,08/03/2023,HCM,Vietnam,Booking.com,WFO,3868.363056,610.311944,2701.667222,815.410000,124.256667,201.734444
1,08/02/2023,HCM,Vietnam,Booking.com,WFO,3893.729167,613.019444,2671.177778,1003.501111,170.857500,203.371944
0,08/01/2023,HCM,Vietnam,Booking.com,WFO,3784.641667,600.958611,2578.053889,756.193056,194.241111,196.090000
154,01/02/2024,HCM,Vietnam,Booking.com,WFO,14.813611,0.000556,8.797500,45.873333,0.807500,0.000556


In [13]:
# Calculate schedule

agent_wise_schedule = schedule


agent_wise_schedule['Workshift'] = (agent_wise_schedule.apply(lambda x: 1 if x['Shift'] != 'OFF' and x['Shift'] != 'AL' and x['Shift'] != 'CO' and x['Shift'] != 'VGH' else 0, axis=1)
+ ((agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'HAL' else 0, axis=1))/2)).astype(float)
agent_wise_schedule['Present for calculation'] = (agent_wise_schedule['Workshift'] - agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'UPL' else 0, axis=1) - (agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'HAL' else 0, axis=1))/2).astype(float)
agent_wise_schedule['Schedule'] = (agent_wise_schedule['Workshift'] + agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'UPL' else 0, axis=1) + (agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'HAL' else 0, axis=1))/2).astype(float)
agent_wise_schedule['Present'] = agent_wise_schedule['Present for calculation']
agent_wise_schedule['Planned Leave'] = agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'AL' or x['Shift'] == 'CO' or x['Shift'] == 'VGH' else 0, axis=1).astype(float)
agent_wise_schedule['Unplanned Leave'] = (agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'UPL' else 0, axis=1) + (agent_wise_schedule.apply(lambda x: 1 if x['Shift'] == 'HAL' else 0, axis=1))/2).astype(float) 


agent_wise_schedule = agent_wise_schedule.groupby(['Date'], as_index=False)[['Workshift','Present for calculation','Schedule','Present','Planned Leave','Unplanned Leave']].sum()
agent_wise_schedule = agent_wise_schedule.sort_values(by=['Date'], ascending=False)



agent_wise_schedule['Date'] = pd.to_datetime(agent_wise_schedule['Date'], format = 'mixed').dt.strftime('%m/%d/%Y')


# agent_wise_schedule.to_json(os.path.join(r'C:', user_credential, r'DB\filejson\agentwisesrp.json'), orient='records')
# agent_wise_schedule.to_excel(os.path.join(r'C:', user_credential, r'DB\filecsv\agentwisesrp.xlsx'),sheet_name='sheet_1')

In [14]:
agent_wise = pd.merge(agent_wise_frame, agent_wise_schedule, left_on='Session Date', right_on='Date', how='left')
agent_wise = agent_wise[['Session Date','Location','Country','Account','Work Status','Staff hour (In hours)','Total break (In hours)','THT (In hours)','Avail Time(In Hours)','Other Productive Aux(In Hours)','Non Productive Aux(In Hours)','Workshift','Present for calculation','Schedule','Present','Planned Leave','Unplanned Leave']]

agent_wise.to_json(os.path.join(r'C:', user_credential, r'DB\filejson\agentwise.json'), orient='records')
agent_wise.to_excel(os.path.join(r'C:', user_credential, r'DB\filecsv\agentwise.xlsx'),sheet_name='sheet_1')

In [15]:
#agent wise from daily tracking

# add session date in EPS
aw_dt_frame_eps = eps

aw_dt_frame_eps['Session login date'] = pd.to_datetime(aw_dt_frame_eps['Session login date'], format='mixed').dt.date
aw_dt_frame_eps['Session login date-1'] = aw_dt_frame_eps['Session login date'] - pd.Timedelta(days=1)
aw_dt_frame_eps['Session login date-1'] = pd.to_datetime(aw_dt_frame_eps['Session login date-1'], format = "mixed")


aw_dt_frame_eps = pd.merge(aw_dt_frame_eps, schedule[['Emp ID','Name','Date','Shift','Shift_type','LOB']], left_on=['EID', 'Session login date-1'], right_on=['Emp ID', 'Date'], how='left')

def session_date(x):
    if x['Shift_type'] == 'NS' and x['Session login time'] <= t(12, 0, 0):
        return x['Session login date-1']
    else:
        return x['Session login date']

aw_dt_frame_eps['Session login time'] = pd.to_datetime(aw_dt_frame_eps['Session login time'], format='mixed').dt.time    
aw_dt_frame_eps['Session date'] = aw_dt_frame_eps.apply(session_date, axis=1)
aw_dt_frame_eps = aw_dt_frame_eps.rename(columns={'Session date': 'Date EPS'})
aw_dt_frame_eps['Date EPS'] = pd.to_datetime(aw_dt_frame_eps['Date EPS'], format = 'mixed')


aw_dt_frame_eps['Duration'] = aw_dt_frame_eps['Total Time']/86400
aw_dt_frame_eps['Productive hours'] = aw_dt_frame_eps.apply(lambda x: x['Duration'] if x['BPE Code'] == 'Picklist - off Phone' or x['BPE Code'] == 'Ready or Talking' else 0, axis=1).astype(float)
aw_dt_frame_eps['Down Time'] = aw_dt_frame_eps.apply(lambda x: x['Duration'] if x['BPE Code'] == 'Meeting' or x['BPE Code'] == 'Training' else 0, axis=1).astype(float)
aw_dt_frame_eps['Delivered hours'] = aw_dt_frame_eps['Productive hours'] + aw_dt_frame_eps['Down Time']
# aw_dt_frame_eps['% Down time'] = aw_dt_frame_eps['Down time']/aw_dt_frame_eps['Delivered hours']
aw_dt_frame_eps['Session Duration'] = aw_dt_frame_eps.apply(lambda x: x['Duration'], axis=1).astype(float)
aw_dt_frame_eps['Staffed Hours'] = aw_dt_frame_eps.apply(lambda x: x['Session Duration'], axis=1).astype(float)


# aw_dt_frame_eps = aw_dt_frame_eps[['Date EPS','EID','Name','BPE Code','Total Time','Duration','Productive hours','Down Time','Delivered hours','Session Duration','Staffed Hours']]
aw_dt_frame_eps = aw_dt_frame_eps.groupby(['Date EPS','EID','Name','Shift','LOB'], as_index=False)[['Total Time','Duration','Productive hours','Down Time','Delivered hours','Session Duration','Staffed Hours']].sum()




aw_dt_frame_eps = pd.merge(aw_dt_frame_eps, masterroster, left_on='EID', right_on='Employee_ID', how='left')
aw_dt_frame_eps['Day Name'] = aw_dt_frame_eps['Date EPS'].dt.strftime('%A')
aw_dt_frame_eps = pd.merge(aw_dt_frame_eps, tl, left_on='EID', right_on='Emp ID', how='left')
# aw_dt_frame_eps = aw_dt_frame_eps[['Date EPS','Day Name','EID','Booking Login ID','Wave #','Supervisor','LOB','Shift','Total Time','Duration','Productive hours','Down Time','Delivered hours','Session Duration','Staffed Hours']]



aw_dt_cpi = cpi
aw_dt_cpi['Phone Case #'] = aw_dt_cpi.apply(lambda x: x['Number of Records'] if x['Channel'] == 'phone' else 0, axis=1)
aw_dt_cpi['Non-Phone Case #'] = aw_dt_cpi.apply(lambda x: x['Number of Records'] if x['Channel'] != 'phone' else 0, axis=1)
aw_dt_cpi = aw_dt_cpi.groupby(['Staff Name','Employee_ID','Date'], as_index=False)[['Phone Case #','Non-Phone Case #']].sum()




aw_dt = pd.merge(aw_dt_frame_eps, aw_dt_cpi, left_on=['EID','Date EPS'], right_on=['Employee_ID','Date'], how='left')
aw_dt['Date EPS'] = pd.to_datetime(aw_dt['Date EPS'], format = 'mixed').dt.strftime('%m/%d/%Y')
aw_dt = aw_dt[['Date EPS','Day Name','EID','Name','Booking Login ID','Wave #','Supervisor','LOB','Shift','Total Time','Duration','Productive hours','Down Time','Delivered hours','Session Duration','Staffed Hours','Phone Case #','Non-Phone Case #']]



# aw_dt.to_excel(os.path.join(r'C:', user_credential, r'DB\filecsv\aw_dt.xlsx'),sheet_name='sheet_1')
aw_dt.to_json(os.path.join(r'C:', user_credential, r'DB\filejson\aw_dt.json'), orient='records')                     